# Solución etapa 4 - FE + Training

In [1]:
# Utilidades para print
from utils.print_utils import tabl, headr, titl

In [2]:
# Cargar los datasets
import pandas as pd

historicos_ordenes_cleaned = pd.read_csv('../data/cleaned/historicos_ordenes_cleaned.csv')
registros_condiciones_cleaned = pd.read_csv('../data/cleaned/registros_condiciones_cleaned.csv')
caracteristicas_equipos_cleaned = pd.read_csv('../data/cleaned/caracteristicas_equipos_cleaned.csv')

## Feature eng

### Target

In [3]:
# Generar target

target_column="Fallo"

def code_target(val):
    if val == 'Correctivo':
        return 1
    else:
        return 0


historicos_ordenes_cleaned[target_column] = historicos_ordenes_cleaned["Tipo_Mantenimiento"].apply(code_target)
historicos_ordenes_cleaned.drop("Tipo_Mantenimiento", axis=1, inplace=True)

### historicos_ordenes

In [4]:
tabl(historicos_ordenes_cleaned)

      ID_Orden    ID_Equipo  Fecha         Costo_Mantenimiento    Duracion_Horas  Ubicacion      Fallo
--  ----------  -----------  ----------  ---------------------  ----------------  -----------  -------
 0           1            1  2021-01-09                   3992                20  Ubicacion_C        1
 1           2            1  2021-01-20                   1689                21  Ubicacion_C        1
 2           3            1  2021-02-02                   2787                 7  Ubicacion_C        1
 3           4            1  2021-02-14                   2857                 9  Ubicacion_C        1
 4           5            1  2021-02-15                   2421                 2  Ubicacion_C        1
(61726, 7)


In [5]:
historicos_ordenes_cleaned['Fecha'] = pd.to_datetime(historicos_ordenes_cleaned['Fecha'])

# historicos_ordenes_cleaned['Año'] = historicos_ordenes_cleaned['Fecha'].dt.year
# historicos_ordenes_cleaned['Mes'] = historicos_ordenes_cleaned['Fecha'].dt.month
# historicos_ordenes_cleaned['Día'] = historicos_ordenes_cleaned['Fecha'].dt.day

tabl(historicos_ordenes_cleaned)

      ID_Orden    ID_Equipo  Fecha                  Costo_Mantenimiento    Duracion_Horas  Ubicacion      Fallo
--  ----------  -----------  -------------------  ---------------------  ----------------  -----------  -------
 0           1            1  2021-01-09 00:00:00                   3992                20  Ubicacion_C        1
 1           2            1  2021-01-20 00:00:00                   1689                21  Ubicacion_C        1
 2           3            1  2021-02-02 00:00:00                   2787                 7  Ubicacion_C        1
 3           4            1  2021-02-14 00:00:00                   2857                 9  Ubicacion_C        1
 4           5            1  2021-02-15 00:00:00                   2421                 2  Ubicacion_C        1
(61726, 7)


### registros_condiciones

In [6]:
registros_condiciones_cleaned.drop(columns=['ID_Registro'], index=1, inplace=True)

registros_condiciones_cleaned['Fecha']=pd.to_datetime(registros_condiciones_cleaned['Fecha'])
# registros_condiciones_cleaned['Año'] = registros_condiciones_cleaned['Fecha'].dt.year
# registros_condiciones_cleaned['Mes'] = registros_condiciones_cleaned['Fecha'].dt.month
# registros_condiciones_cleaned['Día'] = registros_condiciones_cleaned['Fecha'].dt.day

tabl(registros_condiciones_cleaned)

      ID_Equipo  Fecha                  Temperatura_C    Vibracion_mm_s    Horas_Operativas
--  -----------  -------------------  ---------------  ----------------  ------------------
 0            1  2021-01-01 00:00:00          34.7499          2.13681                  614
 2            1  2021-01-03 00:00:00          56.2006          1.52939                  658
 3            1  2021-01-04 00:00:00          70.5556          3.41101                  664
 4            1  2021-01-05 00:00:00          97.509           0.618757                 683
 5            1  2021-01-06 00:00:00          73.4465          4.64147                  701
(730499, 5)


### Unir registro + hitorico + caracteristicas

In [7]:
# Unir datasets
final_data = pd.merge(registros_condiciones_cleaned, historicos_ordenes_cleaned, on=['ID_Equipo','Fecha'], how='left')
final_data = pd.merge(final_data, caracteristicas_equipos_cleaned, on='ID_Equipo', how='inner')

tabl(final_data)

      ID_Equipo  Fecha                  Temperatura_C    Vibracion_mm_s    Horas_Operativas    ID_Orden    Costo_Mantenimiento    Duracion_Horas    Ubicacion    Fallo  Tipo_Equipo    Fabricante    Modelo      Potencia_kW    Horas_Recomendadas_Revision
--  -----------  -------------------  ---------------  ----------------  ------------------  ----------  ---------------------  ----------------  -----------  -------  -------------  ------------  --------  -------------  -----------------------------
 0            1  2021-01-01 00:00:00          34.7499          2.13681                  614         nan                    nan               nan          nan      nan  Bomba          Fabricante_C  Modelo_6            173                            913
 1            1  2021-01-03 00:00:00          56.2006          1.52939                  658         nan                    nan               nan          nan      nan  Bomba          Fabricante_C  Modelo_6            173                        

In [8]:
final_data.columns

Index(['ID_Equipo', 'Fecha', 'Temperatura_C', 'Vibracion_mm_s',
       'Horas_Operativas', 'ID_Orden', 'Costo_Mantenimiento', 'Duracion_Horas',
       'Ubicacion', 'Fallo', 'Tipo_Equipo', 'Fabricante', 'Modelo',
       'Potencia_kW', 'Horas_Recomendadas_Revision'],
      dtype='object')

In [9]:
# Limpiar columnas accesorias
final_data.drop(columns=['ID_Orden','Costo_Mantenimiento', 'Duracion_Horas', 'Ubicacion'], inplace=True)

tabl(final_data)

      ID_Equipo  Fecha                  Temperatura_C    Vibracion_mm_s    Horas_Operativas    Fallo  Tipo_Equipo    Fabricante    Modelo      Potencia_kW    Horas_Recomendadas_Revision
--  -----------  -------------------  ---------------  ----------------  ------------------  -------  -------------  ------------  --------  -------------  -----------------------------
 0            1  2021-01-01 00:00:00          34.7499          2.13681                  614      nan  Bomba          Fabricante_C  Modelo_6            173                            913
 1            1  2021-01-03 00:00:00          56.2006          1.52939                  658      nan  Bomba          Fabricante_C  Modelo_6            173                            913
 2            1  2021-01-04 00:00:00          70.5556          3.41101                  664      nan  Bomba          Fabricante_C  Modelo_6            173                            913
 3            1  2021-01-05 00:00:00          97.509           0.61875

### Duplicados

In [10]:
# Eliminar duplicados

print(headr('Duplicados final_data:'),final_data.duplicated().sum())
# final_data.drop_duplicates(inplace=True)
# tabl(final_data)


Duplicados final_data:
 0


In [11]:
# Verificamos nulos
print(headr('Nulos final_data:'),final_data.isna().sum())


Nulos final_data:
 ID_Equipo                           0
Fecha                               0
Temperatura_C                       0
Vibracion_mm_s                      0
Horas_Operativas                    0
Fallo                          670023
Tipo_Equipo                         0
Fabricante                          0
Modelo                              0
Potencia_kW                         0
Horas_Recomendadas_Revision         0
dtype: int64


In [12]:
columnas_con_nulos = final_data.columns[final_data.isna().any()].tolist()
columnas_con_nulos

['Fallo']

In [13]:
# Imputamos nulos
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="constant", fill_value=0)

final_data[columnas_con_nulos] = imputer.fit_transform(final_data[columnas_con_nulos])

In [14]:
tabl(final_data)

      ID_Equipo  Fecha                  Temperatura_C    Vibracion_mm_s    Horas_Operativas    Fallo  Tipo_Equipo    Fabricante    Modelo      Potencia_kW    Horas_Recomendadas_Revision
--  -----------  -------------------  ---------------  ----------------  ------------------  -------  -------------  ------------  --------  -------------  -----------------------------
 0            1  2021-01-01 00:00:00          34.7499          2.13681                  614        0  Bomba          Fabricante_C  Modelo_6            173                            913
 1            1  2021-01-03 00:00:00          56.2006          1.52939                  658        0  Bomba          Fabricante_C  Modelo_6            173                            913
 2            1  2021-01-04 00:00:00          70.5556          3.41101                  664        0  Bomba          Fabricante_C  Modelo_6            173                            913
 3            1  2021-01-05 00:00:00          97.509           0.61875

In [15]:
print("% fallos:",
final_data[final_data['Fallo'] == 1].shape[0] / final_data.shape[0] * 100)

% fallos: 5.032873293984686


In [16]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731749 entries, 0 to 731748
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID_Equipo                    731749 non-null  int64         
 1   Fecha                        731749 non-null  datetime64[ns]
 2   Temperatura_C                731749 non-null  float64       
 3   Vibracion_mm_s               731749 non-null  float64       
 4   Horas_Operativas             731749 non-null  int64         
 5   Fallo                        731749 non-null  float64       
 6   Tipo_Equipo                  731749 non-null  object        
 7   Fabricante                   731749 non-null  object        
 8   Modelo                       731749 non-null  object        
 9   Potencia_kW                  731749 non-null  int64         
 10  Horas_Recomendadas_Revision  731749 non-null  int64         
dtypes: datetime64[ns](1), floa

### Volvemos a explorar con columnas codificadas


In [17]:
# Extraer columnas numéricas y categóricas
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(final_data)
categorical_columns = categorical_columns_selector(final_data)

print(headr("Numerical columns"), numerical_columns)
print(headr("Categorical columns"), categorical_columns)


Numerical columns
 ['ID_Equipo', 'Fecha', 'Temperatura_C', 'Vibracion_mm_s', 'Horas_Operativas', 'Fallo', 'Potencia_kW', 'Horas_Recomendadas_Revision']

Categorical columns
 ['Tipo_Equipo', 'Fabricante', 'Modelo']


In [18]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

final_data_coded = final_data.copy()
final_data_coded[categorical_columns] = final_data_coded[categorical_columns].apply(LabelEncoder().fit_transform)

# Eliminamos fecha para la evaluación
final_data_coded.drop(columns=['Fecha'], inplace=True)

tabl(final_data_coded)

      ID_Equipo    Temperatura_C    Vibracion_mm_s    Horas_Operativas    Fallo    Tipo_Equipo    Fabricante    Modelo    Potencia_kW    Horas_Recomendadas_Revision
--  -----------  ---------------  ----------------  ------------------  -------  -------------  ------------  --------  -------------  -----------------------------
 0            1          34.7499          2.13681                  614        0              0             2         6            173                            913
 1            1          56.2006          1.52939                  658        0              0             2         6            173                            913
 2            1          70.5556          3.41101                  664        0              0             2         6            173                            913
 3            1          97.509           0.618757                 683        0              0             2         6            173                            913
 4        

In [19]:
# Evaluamos la aportación de cada columna
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif,chi2

X= final_data_coded.drop(target_column, axis=1)
y = final_data_coded[target_column]

fvalue_selector = SelectKBest(f_classif, k=2)

X_kbest = fvalue_selector.fit(X,y)

feature_scores = pd.DataFrame({"Feature": X.columns,"Score": X_kbest.scores_}).sort_values(by="Score", ascending=False)

print(headr("Feature scores"))
round(feature_scores,2)


Feature scores



,Feature,Score
3,Horas_Operativas,40714.39
0,ID_Equipo,4.12
2,Vibracion_mm_s,3.45
6,Modelo,0.95
7,Potencia_kW,0.75
4,Tipo_Equipo,0.55
8,Horas_Recomendadas_Revision,0.48
5,Fabricante,0.07
1,Temperatura_C,0.01


> Una característica tiene demasiado peso y determina toda la clasificación

In [20]:
# Eliminamos columnas sin peso

# columns_to_drop = feature_scores[feature_scores['Score'] < 0.1]['Feature']
# final_data.drop(columns=columns_to_drop, inplace=True)
# tabl(final_data)

## Exportar 

In [21]:
final_data.to_csv('../data/preprocessed/preprocessed_data_v1.csv', index=False)